In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
sns.set_style('whitegrid')
%matplotlib inline


In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import SnowballStemmer, WordNetLemmatizer 
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report, precision_recall_fscore_support, precision_score, recall_score, f1_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Embedding,Dropout, Layer ,LSTM, SpatialDropout1D,Bidirectional,GlobalMaxPool1D, Input
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping 
import keras.backend as K
from tensorflow.keras.optimizers import Adam
import keras
import multiprocessing
nthreads = multiprocessing.cpu_count()
import spacy
nlp = spacy.load('en_core_web_sm')
import string
punctuations = string.punctuation
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import tensorflow as tf
import string
import re
import pandas as pd
import spacy
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import GaussianNB , MultinomialNB
from collections import Counter
pd.set_option('display.max_rows', 104)
pd.set_option('display.max_columns',104)

In [4]:
df = pd.read_csv('/kaggle/input/bonding/hm_train.csv')
df_train = df
test = pd.read_csv('/kaggle/input/bonding/hm_test.csv')

## Data Preprocessing
Data cleaning, lemmatization, and stopwords removal

In [5]:
def clean(data):
    datarm=[]
    for i in data['cleaned_hm']:
        punct = re.sub(r'\W', ' ', i)
        non_num= re.sub ('[0-9]', '', punct)
        multiple_spaces = re.sub(' +',' ',non_num)
        newlines = str(multiple_spaces).replace("\n","")
        returns = str(newlines).replace("\r","")
        quotes = str(returns).replace("'","")
        datarm.append(quotes)
    return datarm

In [6]:
def lemmatize(data):
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    data_lemma = []
    for i in data:
        text = nlp(i)
        data_lemma.append([token.lemma_.lower().strip() for token in text])

    return data_lemma

In [7]:
def remove_stopwords(data):
    en_stop = nltk.corpus.stopwords.words('english')
    for sentence in data:
        for token in sentence:
            if token.lower() in en_stop:
                sentence.remove(token)
    data_stopped = [' '.join(i).lower() for i in data]   
    return data_stopped

In [8]:
def stem(data):
    data_stem = []
    stemmer = PorterStemmer()
    for sentence in data:
        #print(sentence)
        data_stem.append([stemmer.stem(token) for token in sentence.split()])
    return data_stem

In [9]:
def tokenize(data):
    tokenized = []
    for i in data:
        tokenized.append(i.split())
    return tokenized

In [10]:
def preprocess_data(data):
    cleaned=clean(data)
    lemmatized=lemmatize(cleaned)
    #stemmed = stem(cleaned)
    data_cleaned=remove_stopwords(lemmatized)
    #data_cleaned = remove_commonwords(data_cleaned)
    return data_cleaned
    #return stemmed

In [11]:
X_train=preprocess_data(df_train)

In [ ]:
y_train=df_train['predicted_category']
label_encoder = LabelEncoder()
y_encode = label_encoder.fit_transform(y_train)
labels = label_encoder.classes_
y = y_encode

In [14]:
MAX_NB_WORDS = 150000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [15]:
X = tokenizer.texts_to_sequences(X_train)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

## Train test split

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.10, random_state = 42)
x_train.shape, y_train.shape

## BiLSTM Model

In [17]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(100, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(labels), activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 5
batch_size = 128

history = model.fit(x_train, y_train, epochs=epochs,
                    batch_size=batch_size,validation_split=0.2,
                   verbose =1
                   )